In [38]:
import os
import pywt
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from scipy.stats import norm
import numpy.polynomial.polynomial as poly

In [39]:
def import_file(path, limit_low=None, limit_high=None):

    spectrum = np.genfromtxt(path, delimiter=",")
    spectrum = np.transpose(spectrum)
    wavenumbers = spectrum[0]
    intensities = spectrum[1]

    if limit_low is not None:
        limit_low_index = list(wavenumbers).index(limit_low)
    else:
        limit_low_index = 0
        limit_low = wavenumbers[0]

    if limit_high is not None:
        limit_high_index = list(wavenumbers).index(limit_high)
    else:
        limit_high_index = len(wavenumbers)
        limit_high = wavenumbers[-1]

    wavenumbers = wavenumbers[limit_low_index:limit_high_index]
    intensities = intensities[limit_low_index:limit_high_index]
    return wavenumbers, intensities

def import_directory(path, limit_low=None, limit_high=None):
    # files = os.listdir(path)

    # for filename in files:
    #     np.genfromtxt(filename, delimiter=",")
    pass

In [40]:
wavenumbers, intensities = import_file("spectra/E (1).TXT", 300, 1600)

In [45]:
intensities_sg2 = sig.savgol_filter(intensities, 
                                    window_length=25, 
                                    polyorder=3, 
                                    deriv=2)

In [46]:
intensities_sg3 = sig.savgol_filter(intensities, 
                                    window_length=25, 
                                    polyorder=3, 
                                    deriv=3)

In [52]:
zeros = np.diff(np.sign(intensities_sg3))

In [119]:
peaks = np.where(zeros > 0)[0]
peaks = peaks[intensities_sg2[peaks] < -1]

In [136]:
peak_condensing = []
peaks_new = []
for i in range(len(wavenumbers)-1):
    if i in peaks:
        peak_condensing.append(i)
    if intensities_sg2[i] > 0 and len(peak_condensing) > 0:
        peaks_new.append(int(np.mean(peak_condensing)))
        peak_condensing = []
if len(peak_condensing) > 0:
    peaks_new.append(int(np.mean(peak_condensing)))
peaks_new = np.asarray(peaks_new)            
peaks_new

array([  45,  158,  437,  542,  717,  848,  887, 1046, 1097, 1144, 1213,
       1316, 1403, 1503, 1591, 1669, 1815, 1887, 2039, 2146, 2248, 2289,
       2488, 2556])

In [137]:
def plot_raw_spectra(wavenumbers, intensities1, intensities2, intensities3):
    fig, ax = plt.subplots(3,1)
    ax[0].plot(wavenumbers, intensities1, linewidth = 1)
    ax[1].plot(wavenumbers, intensities2, linewidth = 1)
    ax[2].plot(wavenumbers, intensities3, linewidth = 1)
    for i in range(len(ax)):
        ax[i].set_xlim(wavenumbers[0], wavenumbers[-1])
        ax[i].invert_xaxis()
        ax[i].grid()
        if i != 0:
            ax[i].hlines(0, wavenumbers[0], wavenumbers[-1], linewidth=1, color="black")
    ax[0].vlines(wavenumbers[peaks_new], 0, np.max(intensities)*1.1, linewidth=0.7, color="black")
    ax[0].set_ylim(np.min(intensities)*0.9, np.max(intensities)*1.1)

In [138]:
%matplotlib
plot_raw_spectra(wavenumbers, intensities, intensities_sg2, intensities_sg3)

Using matplotlib backend: Qt5Agg
